In [ ]:

import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
pd.set_option('max_rows', None)

In [ ]:

# Loading all datasets and defining directories
for dirname, _, filenames in os.walk('../input/titanic'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

train_df = pd.read_csv('../input/titanic/train.csv')
test_df = pd.read_csv('../input/titanic/test.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
def plot_missing_data(dataset, title):
    fig, ax = plt.subplots(figsize=(5,5))
    plt.title(title)  
    sns.heatmap(dataset.isnull(), cbar=False)
train_df.info()

In [ ]:
plot_missing_data(train_df, "Training Dataset")

In [ ]:
test_df.info()

In [ ]:
plot_missing_data(test_df, "Test Dataset")


In [ ]:
def bar_chart_stacked(dataset, feature, stacked = True):
    survived = dataset[dataset['Survived']==1][feature].value_counts()
    dead = dataset[dataset['Survived']==0][feature].value_counts()
    df_survived_dead = pd.DataFrame([survived,dead])
    df_survived_dead.index = ['Passengers Survived','Passengers Died']
    ax = df_survived_dead.plot(kind='bar',stacked=stacked, figsize=(5,5))

In [ ]:
train_df['Survived'].value_counts()

In [ ]:
train_df['Survived'].value_counts(normalize=True)

In [ ]:
bar_chart_stacked(train_df, "Survived")


In [ ]:
train_df['Sex'].value_counts().to_frame()

In [ ]:
bar_chart_stacked(train_df, "Sex")

In [ ]:
train_df.groupby('Sex').Survived.mean()

In [ ]:
bar_chart_stacked(train_df, 'Pclass')

In [ ]:
pd.pivot_table(train_df, index = 'Survived', columns = 'Pclass', values = 'Ticket' ,aggfunc ='count')
train_df.groupby(['Pclass']).Survived.mean().to_frame()
def bar_chart_compare(dataset, feature1, feature2=None, title = "Survival rate by sex and class'"):
    plt.figure(figsize = [5,5])
    plt.title(title)
    g = sns.barplot(x=feature1, y='Survived', hue=feature2, ci=None, data=dataset).set_ylabel('Survival rate')

In [ ]:
bar_chart_compare(train_df, "Pclass", "Sex")

In [ ]:
pd.pivot_table(train_df, index = 'Survived', columns = ['Pclass', "Sex"], values = 'Ticket' ,aggfunc ='count')

In [ ]:
train_df.groupby(['Pclass', "Sex"]).Survived.mean().to_frame()

In [ ]:
def plot_distribution(dataset, feature, title, bins = 30, hist = True, fsize = (5,5)):
    fig, ax = plt.subplots(figsize=fsize)
    ax.set_title(title)
    sns.distplot(train_df[feature], color='g', bins=bins, ax=ax)

In [ ]:
def plot_kernel_density_estimate_survivors(dataset, feature1, title, fsize = (5,5)):
    fig, ax = plt.subplots(figsize=fsize)
    ax.set_title(title) 
    sns.kdeplot(dataset[feature1].loc[train_df["Survived"] == 1],
                shade= True, ax=ax, label='Survived').set_xlabel(feature1)
    sns.kdeplot(dataset[feature1].loc[train_df["Survived"] == 0],
                shade=True, ax=ax, label="Died")

In [ ]:
plot_distribution(train_df, "Age", "Age Distribution Passengers")


In [ ]:
plot_kernel_density_estimate_survivors(train_df, "Age", "Age Distribution Surived vs Died")

In [ ]:

def plot_swarm_survivors(dataset, feature1, feature2, title, fize = (155)):
    fig, ax = plt.subplots(figsize=(18,5))
    # Turns off grid on the left Axis.
    ax.grid(True)
    plt.xticks(list(range(0,100,2)))
    sns.swarmplot(y=feature1, x=feature2, hue='Survived',data=train_df).set_title(title)

In [ ]:
plot_swarm_survivors(train_df, "Sex", "Age", "Survivor Swarmplot for Age and Gender")

In [ ]:
plot_swarm_survivors(train_df, "Age", "Pclass", "Survivor Swarmplot for Age and Gender")

In [ ]:
train_df.Fare.describe()


In [ ]:
plot_distribution(train_df, "Fare", "Fare Distribution Passengers")

In [ ]:

def plot_quartiles(dataset, feature, title, categories):
    fig, axarr = plt.subplots(figsize=(5,5))
    fare_ranges = pd.qcut(dataset[feature], len(categories), labels = categories) #. [0, .25, .5, .75, 1.]
    axarr.set_title(title)
    sns.barplot(x=fare_ranges, y=dataset.Survived, ci=None, ax=axarr).set_ylabel('Survival rate')

In [ ]:
categories = ['Cheap', 'Standard', 'Expensive', 'Luxury']

plot_quartiles(train_df, "Fare", "Survival Rate by Fare Ranges/Categories", categories)

In [ ]:
plot_swarm_survivors(train_df, "Fare", "Sex","Survivor Swarmplot for Age and Gender")

In [ ]:
train_df.loc[train_df.Fare==0]

In [ ]:
len(train_df.loc[train_df.Fare==0])

In [ ]:
# Replace Fare == 0 with nan
train_df.loc[train_df['Fare'] == 0, 'Fare'] = np.NaN
test_df.loc[train_df['Fare'] == 0, 'Fare'] = np.NaN

In [ ]:

def show_countplot(dataset, feature, title, fsize = (5,5)):
    fig, ax = plt.subplots(figsize=fsize)
    sns.countplot(dataset[feature], ax=ax).set_title(title)
    
def show_compare_countplot(dataset, feature1, feature2, title):
    fig, ax = plt.subplots(figsize=(5,5))
    p = sns.countplot(x = feature1, hue = feature2, data = dataset, ax=ax).set_title(title)

In [ ]:
bar_chart_stacked(train_df, 'Embarked')


In [ ]:
show_countplot(train_df, "Embarked", 'Passengers count by boarding point')

In [ ]:
train_df['Embarked'].value_counts().to_frame()

In [ ]:
show_compare_countplot(train_df, "Embarked", "Survived", "Survivor count by place of embarktion")

In [ ]:
pd.pivot_table(train_df, index = 'Survived', columns = 'Embarked', values = 'Ticket' ,aggfunc ='count')

In [ ]:
train_df.groupby(['Embarked']).Survived.mean().to_frame()

In [ ]:
show_compare_countplot(train_df, "Embarked", "Pclass", "Passenger count by place of embarktion and class")

In [ ]:
train_df.groupby(['Embarked', 'Pclass']).Survived.sum().to_frame()

In [ ]:
show_compare_countplot(train_df, "Embarked", "Sex", "Passenger count by place of embarktion and sex")

In [ ]:
train_df['SibSp'].value_counts().to_frame()

In [ ]:
bar_chart_compare(train_df, "SibSp", title = "Surival rate by siblings / spouses aboard the Titanic")

In [ ]:
train_df.groupby(['SibSp']).Survived.mean().to_frame()

In [ ]:
show_countplot(train_df, "SibSp", 'SibSp Distribution', (15,3))

In [ ]:
show_compare_countplot(train_df, "SibSp", "Survived", "Survivor count by number of siblings / spouses aboard the Titanic")

In [ ]:
show_countplot(train_df, "Parch", 'Parch Distribution', (15,3))

In [ ]:
bar_chart_compare(train_df, "Parch", title = "Survival rate by Parch")

In [ ]:
train_df.groupby(['Parch']).Survived.mean().to_frame()

In [ ]:
show_compare_countplot(train_df, "Parch", "Survived", "Survivor count by Parch")

In [ ]:
pd.unique(train_df['Name'])

In [ ]:
train_df['Title'] = train_df['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
test_df['Title'] = test_df['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())

In [ ]:
train_df.head()

In [ ]:
train_df['Title'].value_counts().to_frame()

In [ ]:
show_countplot(train_df, "Title", 'Title Distribution', (15,3))

In [ ]:
train_df['Title'].replace(['Mme', 'Ms', 'Lady', 'Mlle', 'the Countess', 'Dona'], 'Miss', inplace=True)
test_df['Title'].replace(['Mme', 'Ms', 'Lady', 'Mlle', 'the Countess', 'Dona'], 'Miss', inplace=True)
train_df['Title'].replace(['Major', 'Col', 'Capt', 'Don', 'Sir', 'Jonkheer'], 'Mr', inplace=True)
test_df['Title'].replace(['Major', 'Col', 'Capt', 'Don', 'Sir', 'Jonkheer'], 'Mr', inplace=True)

In [ ]:
show_countplot(train_df, "Title", 'Title Distribution after substitution', (15,3))

In [ ]:
bar_chart_stacked(train_df, 'Title')

In [ ]:
bar_chart_compare(train_df, "Title", title = "Survival rate by Title")

In [ ]:
train_df[['Cabin', 'Ticket']]

In [ ]:
train_df['Ticket_2letter'] = train_df.Ticket.apply(lambda x: x[:2])
test_df['Ticket_2letter'] = test_df.Ticket.apply(lambda x: x[:2])

In [ ]:
# Extract Ticket Lenght
train_df['Ticket_len'] = train_df.Ticket.apply(lambda x: len(x))
test_df['Ticket_len'] = test_df.Ticket.apply(lambda x: len(x))

In [ ]:
# Extract Number of Cabins:
train_df['Cabin_num'] = train_df.Ticket.apply(lambda x: len(x.split()))
test_df['Cabin_num'] = test_df.Ticket.apply(lambda x: len(x.split()))

In [ ]:
# Extract Leading Letter:
train_df['Cabin_1letter'] = train_df.Ticket.apply(lambda x: x[:1])
test_df['Cabin_1letter'] = test_df.Ticket.apply(lambda x: x[:1])

In [ ]:
len(train_df['Ticket'].value_counts().to_frame())

In [ ]:
len(train_df["Ticket_2letter"].value_counts().to_frame())

In [ ]:
len(train_df["Ticket_len"].value_counts().to_frame())

In [ ]:
train_df.head()

In [ ]:
len(train_df['Cabin'].value_counts().to_frame())

In [ ]:
len(train_df['Cabin_num'].value_counts().to_frame())

In [ ]:
train_df['Cabin_num'].value_counts().to_frame()

In [ ]:
len(train_df['Cabin_1letter'].value_counts().to_frame())

In [ ]:
train_df['Fam_size'] = train_df['SibSp'] + train_df['Parch'] + 1
test_df['Fam_size'] = test_df['SibSp'] + test_df['Parch'] + 1

In [ ]:
bar_chart_compare(train_df, "Fam_size", title = "Survival rate by family size")

In [ ]:
show_compare_countplot(train_df, "Fam_size", "Survived", "Survivor count by family size")

In [ ]:
# Creation of four groups
train_df['Fam_type'] = pd.cut(train_df.Fam_size, [0,1,4,7,11], labels=['Solo', 'Small', 'Big', 'Very big'])
test_df['Fam_type'] = pd.cut(test_df.Fam_size, [0,1,4,7,11], labels=['Solo', 'Small', 'Big', 'Very big'])

In [ ]:
bar_chart_compare(train_df, "Fam_type", title = "Surival rate by family type")

In [ ]:
show_compare_countplot(train_df, "Fam_type", "Survived", "Survivor count by family type")

In [ ]:
y = train_df['Survived']
features = ['Pclass', 'Fare', 'Title', 'Embarked', 'Fam_type', 'Ticket_len', 'Ticket_2letter']
X = train_df[features]
X.head()

In [ ]:
numerical_cols = ['Fare']
categorical_cols = ['Pclass', 'Title', 'Embarked', 'Fam_type', 'Ticket_len', 'Ticket_2letter']

# Inputing numerical values with median
numerical_transformer = SimpleImputer(strategy='median')

# Inputing missing values with most frequent one for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Bundle preprocessing and modeling code 
titanic_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(random_state=0, n_estimators=500, max_depth=5))
])

# Training
titanic_pipeline.fit(X,y)

print('Cross validation score: {:.3f}'.format(cross_val_score(titanic_pipeline, X, y, cv=10).mean()))

In [ ]:

X_test = test_df[features]
X_test.head()

In [ ]:
predictions = titanic_pipeline.predict(X_test)

In [ ]:
output = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print('Your submission was successfully saved!')

In [ ]:
from IPython.display import HTML
import base64
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)
create_download_link(output)